In [46]:
import pandas as pd
import requests
import datetime
import os
from bs4 import BeautifulSoup

players_path = "webs"

In [61]:
def get_transfers(soup):
    transfers = []
    trasnfer_table = soup.find('table', {'class': 'base-table transfer-table'})
    if not trasnfer_table: return transfers
    trs = trasnfer_table.find('tbody').findAll('tr')

    for tr in trs:
        tds = tr.findAll('td')
        date = datetime.datetime.strptime( tds[0].text, "%b %d, %y" )

        from_a = tds[1].find('a')
        from_href = from_a["href"][6:-1].replace("/","_")

        to_a = tds[2].find('a')
        to_href = to_a["href"][6:-1].replace("/","_")

        reason = tds[3].text

        transfers.append([player_id, date, from_href, to_href, reason])
    return transfers

In [64]:
transfers = []
for dirname, dirnames, filenames in os.walk(players_path):
    # print path to all filenames.
    for filename in filenames:
        if filename.endswith(".html"):
            player_id = dirname[5:].replace("/","_")
            transfer_soup:BeautifulSoup = None
                
            with open(os.path.join(dirname, filename), "r") as r:
                transfer_soup = BeautifulSoup(r, "lxml")
            transfers.extend(get_transfers(transfer_soup))
            

transfers_df = pd.DataFrame(transfers, columns=['player_id', 'date', 'from', 'to', 'reason'])
transfers_df       

,player_id,date,from,to,reason
0,menez-jeremy_A10nmDRJ,2018-01-05,antalyaspor_GAtxDHF9,club-america_vyrswtJm,Transfer
1,menez-jeremy_A10nmDRJ,2017-07-01,bordeaux_SKc9FeQ7,antalyaspor_GAtxDHF9,Transfer
2,menez-jeremy_A10nmDRJ,2016-08-01,ac-milan_8Sa8HInO,bordeaux_SKc9FeQ7,Free agent
3,menez-jeremy_A10nmDRJ,2014-07-01,paris-sg_CjhkPw0k,ac-milan_8Sa8HInO,Free agent
4,menez-jeremy_A10nmDRJ,2011-07-25,as-roma_zVqqL0ma,paris-sg_CjhkPw0k,Transfer
5,menez-jeremy_A10nmDRJ,2008-08-27,monaco_2PIvr8o4,as-roma_zVqqL0ma,Transfer
6,menez-jeremy_A10nmDRJ,2006-07-01,sochaux_jPojkLXK,monaco_2PIvr8o4,Transfer
7,jimenez-oscar_tOvSCmQb,2017-01-01,jaguares-de-chiapas_CdS3LvlD,club-america_vyrswtJm,?
8,jimenez-oscar_tOvSCmQb,2013-07-01,lobos-buap_Umprep5l,jaguares-de-chiapas_CdS3LvlD,?
9,jimenez-oscar_tOvSCmQb,2010-01-01,indios_hx4gdV7G,lobos-buap_Umprep5l,?


In [65]:
transfers_df.to_csv("transfers.csv")